# External file download to Stage
This leverages python libraries to download files from external public URLs to a stage. The intent of this is a demonstration asset to show the entire setup/configuration end to end.

##### Step 1: 
Create an Internal Stage/Target to Download \
Also create a table to collect Telemetry Data

In [ ]:
USE ROLE ACCOUNTADMIN;
CREATE EVENT TABLE IF NOT EXISTS event_table;

In [ ]:
USE ROLE SYSADMIN;
CREATE STAGE IF NOT EXISTS SHP_FILES COMMENT = 'General purpose stage for data loading';

## External Network Rules/Access Setup

##### Step 2:
For this we need to setup the values list of network rules of URLs/Hosts and also activate that integration



In [ ]:
USE ROLE ACCOUNTADMIN;
CREATE OR REPLACE NETWORK RULE FEMA_NETWORK_RULE
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = (  
  '*.FEMA.GOV',);

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION FEMA_INTEGRATION
ALLOWED_NETWORK_RULES = (FEMA_NETWORK_RULE)
ENABLED=TRUE;

GRANT USAGE ON INTEGRATION FEMA_INTEGRATION TO ROLE SYSADMIN;

SHOW INTEGRATIONS;
--Note, need to enable the external integration for the notebook

## Download from URL
##### Step 3:
Download the file. This logic can be adjusted for your specific use case integration. This also could be executed as a task/job

In [ ]:
import requests
import os, sys, datetime
import tempfile
import logging
from snowflake.snowpark.exceptions import SnowparkClientException
session = get_active_session()
current_timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
logging.getLogger('external_url_download')
logger = logging.getLogger('snowflake.snowpark.session')
logger.setLevel(logging.DEBUG)
###### Global Variables ###### 
file_url = "https://msc.fema.gov/portal/downloadProduct?productID=NFHL_48201C" # Replace with the actual URL
target_stage = "@SHP_FILES" # Replace with your Snowflake stage name
target_file_name = f"fema_temp_file{current_timestamp}.zip" # Desired name for the file on the stage





def list_stage_files():
    """Helper function to list files in stage"""
    files = session.sql("LIST @SHP_FILES").collect()
    if len(files) > 0:
        print("Files currently in stage:")
        for file in files:
            print(file['name'])
    else:
        print("No files in stage")



logger.info('Starting Download of File')
print('Starting Download of External URL/File')

# --- Configuration ---

# Use a temporary directory provided by the Snowflake environment
# For Snowpark, /tmp is generally safe for temporary files
temp_dir = tempfile.gettempdir()
local_temp_file_path = os.path.join(temp_dir, target_file_name)


try:
    # 1. Download the file content to a temporary local file
    logger.info(f"Attempting to download file from: {file_url} to {local_temp_file_path}")
    print(f"Attempting to download file from: {file_url} to {local_temp_file_path}")
    with requests.get(file_url, stream=True) as r:
        r.raise_for_status() # Raise an exception for HTTP errors
        with open(local_temp_file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    logger.info(f"Successfully downloaded file to: {local_temp_file_path}")
    print(f"Successfully downloaded file to: {local_temp_file_path}")
    # 2. Upload the temporary local file to the Snowflake stage
    # session.file.put(local_file_name, stage_location, auto_compress, overwrite)
    put_result = session.file.put(
        local_temp_file_path,
        f"{target_stage}/{target_file_name}",
        auto_compress=False,
        overwrite=True
    )

    logger.info(f"File uploaded to stage: {target_stage}/{target_file_name}")
    print(f"File uploaded to stage: {target_stage}/{target_file_name}")
    logger.info("Upload result:")
    for res in put_result:
        logger.info(f"  Source: {res.source}, Target: {res.target}, Status: {res.status}, Message: {res.message}")
        #print(f"Source: {res.source}, Target: {res.target}, Status: {res.status}, Message: {res.message}")
        list_stage_files()
    # Optional: Clean up the temporary local file
    os.remove(local_temp_file_path)
    logger.info(f"Cleaned up temporary file: {local_temp_file_path}")

except requests.exceptions.RequestException as e:
    logger.error(f"Error during file download: {e}")
    print(f"Error during file download: {e}")
except SnowparkClientException as e:
    logger.error(f"Error during file upload to Snowflake stage: {e}")
    print(f"Error during file upload to Snowflake stage: {e}")
except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")
    print(f"An unexpected error occurred: {e}")
finally:
    # Ensure temporary file is removed even if errors occur
    if os.path.exists(local_temp_file_path):
        os.remove( )
        logger.info(f"Ensured cleanup of temporary file: {local_temp_file_path}")
        list_stage_files()

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from urllib.parse import urlparse, quote
import os
import uuid
from datetime import datetime
import pandas as pd

# Get the active Snowflake session
session = get_active_session()

# Set page title and add some spacing
st.title("🗂️ File Management System")
st.markdown("---")

# Download New File Section
st.markdown("### 📥 Download New File")
with st.form("download_form"):
    file_url = st.text_input(
        "Enter the URL of the file to download",
        value="https://msc.fema.gov/portal/downloadProduct?productID=NFHL_48201C",
        placeholder="Enter URL here..."
    )
    submit_button = st.form_submit_button("Download and Stage File", type="primary")
    
    if submit_button and file_url:
        with st.spinner("Downloading file..."):
            try:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                original_filename = os.path.basename(urlparse(file_url).path)
                
                if not original_filename:
                    random_uuid = str(uuid.uuid4())[:8]
                    filename = f"file_{random_uuid}_{timestamp}.zip"
                else:
                    name, ext = os.path.splitext(original_filename)
                    random_uuid = str(uuid.uuid4())[:8]
                    filename = f"{name}_{random_uuid}_{timestamp}{ext}"
                
                encoded_url = quote(file_url, safe=':/?=&')
                downloader = SnowflakeFileDownloader(
                    session=session,
                    source_url=file_url,
                    target_stage="@SHP_FILES",
                    target_filename=filename
                )
                if downloader.download_to_stage():
                    st.success("✅ File downloaded successfully")
                    st.rerun()
                    
            except Exception as e:
                st.error(f"❌ Error processing file: {str(e)}")

# Files in Stage Section
st.markdown("---")
st.markdown("### 📁 Files in Stage")

# Get stage files with metadata
stage_files = session.sql("""
    LIST @SHP_FILES
""").collect()

if len(stage_files) > 0:
    # Convert to pandas DataFrame for better display
    df = pd.DataFrame(stage_files)
    
    # Format size to human-readable format
    def format_size(size):
        for unit in ['B', 'KB', 'MB', 'GB']:
            if size < 1024:
                return f"{size:.2f} {unit}"
            size /= 1024
        return f"{size:.2f} TB"
    
    # Format the DataFrame
    display_df = pd.DataFrame({
        'File Name': df['name'].apply(lambda x: x.split('/')[-1]),
        'Size': df['size'].apply(format_size),
        'Last Modified': pd.to_datetime(df['last_modified']).dt.strftime('%Y-%m-%d %H:%M:%S'),
        'Actions': [''] * len(df)  # Placeholder for buttons
    })
    
    # Display the table
    for idx, row in display_df.iterrows():
        with st.container():
            cols = st.columns([3, 1, 2, 2])
            cols[0].write(row['File Name'])
            cols[1].write(row['Size'])
            cols[2].write(row['Last Modified'])
            
            # Action buttons
            button_cols = cols[3].columns(2)
            with button_cols[0]:
                if st.button("📥", key=f"get_{df['name'].iloc[idx]}", help="Download file"):
                    try:
                        result = session.sql(f"GET @SHP_FILES/{df['name'].iloc[idx]}").collect()
                        st.download_button(
                            label="Save File",
                            data=result[0]['FILE_URL'],
                            file_name=df['name'].iloc[idx].split('/')[-1],
                            mime="application/octet-stream",
                            key=f"download_{df['name'].iloc[idx]}"
                        )
                    except Exception as e:
                        st.error(f"Error getting file: {str(e)}")
            
            with button_cols[1]:
                if st.button("🗑️", key=f"delete_{df['name'].iloc[idx]}", help="Delete file"):
                    try:
                        session.sql(f"REMOVE @SHP_FILES/{df['name'].iloc[idx]}").collect()
                        st.success(f"Deleted {df['name'].iloc[idx]}")
                        st.rerun()
                    except Exception as e:
                        st.error(f"Error deleting file: {str(e)}")
    
    # Add a horizontal line after each file
    st.markdown("---")
else:
    st.info("📂 No files in stage")


In [ ]:
list  @SHP_FILES;
